# Studie av tekster 

Her for barnebøker

fra mail

For litt siden skrev du (13. mars):

«Du hadde noen spørsmål knyttet til ordlistene og hva som kan gjøres videre. Når det gjelder geografi, land og byer, har vi en oversikt. Likeså for navn, men ikke hvor navnene kommer fra. Det kan vi se på etter hvert - hvilke fornavn og etternavn fins, og hva forteller dem om kulturelt mangfold? Vi har metoder under utvikling for å få et bedre grep om dem»

Jeg kunne ha god bruk for at disse ble lagt inn, slik at jeg får en oversikt over hvilke geografiske steder utenom Norge som blir nevnt i materialet – og hvor ofte – samt hvilke fornavn og etternavn som blir brukt.

 

13. mars skrev du: «Hvis excel er noe du kan tenke deg å bruke, kan jeg generere hele excelbøker med oversikt over enkeltordene og verkene de finnes i.»

Det høres ut som et veldig nyttig arbeidsverktøy for meg.

 

13. mars skrev du også:

«Mulig det greieste er å ta ut skolebøker som et eget korpus ved hjelp av Sofies lister.»

Jeg skal ikke bruke skolebøkene til noe, det greieste for meg er å vite at de overhode ikke er med i korpuset, så hvis det lar seg gjøre å ta dem ut, ville det være fint.

 

In [1]:
import json
import sqlite3
import requests
import pandas as pd
import dask
import pandas as pd
import dhlab.module_update as mu
mu.update('mods_meta')
import mods_meta as mm
import dhlab.nbtext as nb



Updated file `D:\Documents\GitHub\Miscellaneous-Notebooks\Barnebokinstituttet_prosjekt\mods_meta.py`

In [2]:
def remove_school(meta):
    newmeta = meta[~meta['653- - -a'].str.contains('lærebøker', na=False)]
    newmeta = newmeta[~newmeta['653- - -a'].str.contains('grunnskolen', na=False)]
    return newmeta

In [3]:
def aggregate_corpus(urns, step = 300):
    babi = pd.DataFrame()
    for i in range(0, len(urns), step):
        print(i)
        df = nb.get_aggregated_corpus(urns[i:i+step])
        babi = nb.frame(pd.concat([babi, df], axis=1, sort=False).sum(axis=1))
    return babi

In [19]:
mu.css()

# Henter ut barnebøker fra 2015



In [4]:
barnebøker_fra_2015 = mm.meta_natbib(children=True, yearfrom =2015, limit=1000)

## tar ut URNene

In [5]:
barnebok_urns = [z[1] for z in barnebøker_fra_2015]

In [6]:
barnebok_urns[:10]

[2018062607117,
 2018072507154,
 2018070405001,
 2019011448552,
 2018071807038,
 2018101107110,
 2018061807565,
 2018102307113,
 2018101007316,
 2018030548092]

# Filtrer URNene fra 

Henter metadata, og filterer bort de med subjet grunnskole, læerebøker

In [7]:
metadata_df = mm.metadata(urns = barnebok_urns)

In [8]:
metadata_df

{'2011062708095': {'100-1- -a': 'Holm, Dagny',
  '245-1-0-a': 'Zeppelin',
  '260- - -c': '2007',
  '653- - -a': 'grunnskolen, k06, lesebøker, lærebøker, mellomtrinnet, norsk',
  'lang': 'nno',
  'year': 2016},
 '2014111907535': {'100-1- -a': 'Holm, Dagny',
  '245-1-0-a': 'Zeppelin',
  '260- - -c': '2007',
  '653- - -a': 'grunnskolen, k06, lesebøker, lærebøker, norsk',
  'lang': 'nno',
  'year': 2015},
 '2015090906138': {'100-1- -a': 'Daynes, Katie.',
  '260- - -c': '2015',
  '653- - -a': 'barnebøker, drager, draker, lettlest',
  'lang': 'nno',
  'year': 2015},
 '2015091006002': {'100-1- -a': 'Punter, Russell.',
  '260- - -c': '2015',
  '653- - -a': 'barnebøker, lettlest',
  'lang': 'nno',
  'year': 2015},
 '2015091006003': {'100-1- -a': 'Punter, Russell.',
  '260- - -c': '2015',
  '653- - -a': 'barnebøker, katter, lettlest',
  'lang': 'nno',
  'year': 2015},
 '2015091006004': {'100-1- -a': 'Likar, Gudrun.',
  '245-1-0-a': 'Prinsesse Fibi og draken',
  '260- - -c': '2015',
  '653- - -a'

In [9]:
meta_bulitt = remove_school(nb.frame(metadata_df))

Sån ser metadataene ut, header er MARC21-koder

In [11]:
meta_bulitt.head(20)

,100-1- -a,245-1-0-a,260- - -c,653- - -a,lang,year,245-1-0-b,245-1-3-a,245-1-3-b
2015090906138,"Daynes, Katie.",NaN,2015,"barnebøker, drager, draker, lettlest",nno,2015,NaN,NaN,NaN
2015091006002,"Punter, Russell.",NaN,2015,"barnebøker, lettlest",nno,2015,NaN,NaN,NaN
2015091006003,"Punter, Russell.",NaN,2015,"barnebøker, katter, lettlest",nno,2015,NaN,NaN,NaN
2015091006004,"Likar, Gudrun.",Prinsesse Fibi og draken,2015,"barnebøker, drager, draker, lettlest, prinsesser",nno,2015,NaN,NaN,NaN
2015091006005,"Berge, Atle",Matias Maradona,2015,"barnebøker, fotball, lettlest",nno,2015,NaN,NaN,NaN
2015091506021,"Muszynski, Eva",Cowboy-Claus og den prompande ponnien,2015,"barnebøker, lettlest",nno,2015,NaN,NaN,NaN
2015091506022,"Mackinnon, Mairi.",Sola og vinden :,2015,"barnebøker, fabler, lettlest, æsop",nno,2015,basert på ein fabel av Æsop,NaN,NaN
2015091506023,"Heinlein, Sylvia",NaN,2015,"barnebøker, lettlest",nno,2015,NaN,NaN,NaN
2015091506024,"Markhus, Vegard",Epleslang,2015,"barnebøker, barnelitteratur, lettlest, norsk",nno,2015,NaN,NaN,NaN
2015091506131,"Baltscheit, Martin.","Felline, professor Pål og kjemisettet",2015,NaN,nno,2015,NaN,NaN,NaN


In [107]:
meta_bulitt.to_excel('metadata_bulitt_2015.xls')

# Henter ut all tekst

et par step_

* hent URNer fra bulittdata
* hent tekst fra NB-digital

Teksten er frekvenslister fra hver bok, og de danner en dokument-term-matrise. Hver kolonne er en bok, og radene er distribusjone for et ord i korpuset. 

In [12]:
barnebok_urns = [str(x) for x in meta_bulitt.index]

In [13]:
corpus_text = nb.get_corpus_text(barnebok_urns)

In [32]:

corpus_text.sort_values(by=corpus_text.columns[4], ascending=False).iloc[range(0,20),range(0,5)].fillna(0).style.background_gradient()

,2015090906138,2015091006002,2015091006003,2015091006004,2015091006005
.,102,38,97,268,241
«,29,4,38,85,108
»,29,2,43,83,107
",",51,27,49,120,91
Matias,0,0,0,0,74
på,5,6,6,27,52
og,32,23,32,74,44
!,10,3,14,32,44
han,11,1,15,18,43
er,17,12,25,26,40


In [108]:
corpus_text.to_csv('corpus_text_document_term_matrix.csv')

# Her ser vi på ordlister


Henter ut ordlister fra mail

fra mail:  
Handikappet/funksjonshemmet
Blind, svaksynt, tykk, tynn, døv, tunghørt, hørselshemmet, klumpfot, albino
 
Seksuell legning
trans, transe, homofil, homse, lesbisk, lesbe, bi, bifil
 
Egennavn
Fornavn, etternavn
 
Nasjonalitet
Pakistansk, arabisk, afrikansk, grønlender, grønlandsk, islandsk, polsk, russer, russisk, amerikaner, hindu, japaner, asiat
Finnes det en base som angir nasjonalitet?
 
Geografisk område
Norden, Europa, Asia, Amerika, Afrika, Australia, Antarktis, Arktis, Ural
Finnes det en base over land? Men dette går kanskje under kategorien Egennavn?
 
Språk – pt kommer jeg på disse, men dersom du har en base med flere språk sier jeg ikke nei til det:
engelsk, pakistansk, urdu, svensk, dansk, finsk, islandsk, grønlandsk, samisk, fransk, tysk, polsk, litauisk, russisk, italiensk, spansk, castillano, portugisisk, kinesisk, japansk, tamil, arabisk, swahili
 
Religion
kristendom, hinduisme, islam, muhammedanisme, taoisme, jødedom, koptisk, katolisisme
 
Andre kultur-markører
Urfolk, indianer, araber, afrikaner, amerikaner, europeer, nordisk, samisk
 

In [33]:
ordlister = """ 
diff: blind, svaksynt, tykk, tynn, døv, tunghørt, hørselshemmet, klumpfot, albino;
seksualitet: trans, transe, homofil, homse, lesbisk, lesbe, bi, bifil;
nasjonalitet: pakistansk, arabisk, afrikansk, grønlender, grønlandsk, islandsk, polsk, russer, russisk, amerikaner, hindu, japaner, asiat;
geografi: Norden, Europa, Asia, Amerika, Afrika, Australia, Antarktis, Arktis, Ural;
språk: engelsk, pakistansk, urdu, svensk, dansk, finsk, islandsk, grønlandsk, samisk, fransk, tysk, polsk, litauisk, russisk, italiensk, spansk, castillano, portugisisk, kinesisk, japansk, tamil, arabisk, swahili;
religion: kristendom, hinduisme, islam, muhammedanisme, taoisme, jødedom, koptisk, katolisisme;
kulturmarkør: urfolk, indianer, araber, afrikaner, amerikaner, europeer, nordisk, samisk;
"""

In [34]:
ordgrupper = {x.split(':')[0]:x.split(':')[1].split(', ') for x in [y.strip() for x in ordlister.split('\n') for y in x.split(';') if y.strip() != '']}

In [35]:
ordgrupper.keys()

dict_keys(['diff', 'seksualitet', 'nasjonalitet', 'geografi', 'språk', 'religion', 'kulturmarkør'])

sjekk at alle ordene ikke har none blanke tegn rundt seg

In [39]:
temaer = {key: [word.strip() for word in ordgrupper[key]] for key in ordgrupper}

In [40]:
temaer

{'diff': ['blind',
  'svaksynt',
  'tykk',
  'tynn',
  'døv',
  'tunghørt',
  'hørselshemmet',
  'klumpfot',
  'albino'],
 'seksualitet': ['trans',
  'transe',
  'homofil',
  'homse',
  'lesbisk',
  'lesbe',
  'bi',
  'bifil'],
 'nasjonalitet': ['pakistansk',
  'arabisk',
  'afrikansk',
  'grønlender',
  'grønlandsk',
  'islandsk',
  'polsk',
  'russer',
  'russisk',
  'amerikaner',
  'hindu',
  'japaner',
  'asiat'],
 'geografi': ['Norden',
  'Europa',
  'Asia',
  'Amerika',
  'Afrika',
  'Australia',
  'Antarktis',
  'Arktis',
  'Ural'],
 'språk': ['engelsk',
  'pakistansk',
  'urdu',
  'svensk',
  'dansk',
  'finsk',
  'islandsk',
  'grønlandsk',
  'samisk',
  'fransk',
  'tysk',
  'polsk',
  'litauisk',
  'russisk',
  'italiensk',
  'spansk',
  'castillano',
  'portugisisk',
  'kinesisk',
  'japansk',
  'tamil',
  'arabisk',
  'swahili'],
 'religion': ['kristendom',
  'hinduisme',
  'islam',
  'muhammedanisme',
  'taoisme',
  'jødedom',
  'koptisk',
  'katolisisme'],
 'kulturmarkør

# DF og TF

In [48]:
check_theme = lambda word, corpus = corpus_text: corpus.loc[[w for w in temaer[word] if w in corpus.index]].count(axis=1)

In [49]:
check_theme('seksualitet')

trans       2
transe     32
homofil    13
homse       8
lesbisk     7
lesbe       3
bi         48
bifil       2
dtype: int64

In [53]:
meta_bulitt.loc[[str(x) for x in corpus_text.loc[check_theme('seksualitet').index].dropna(axis=1, how='all').columns]]

,100-1- -a,245-1-0-a,260- - -c,653- - -a,lang,year,245-1-0-b,245-1-3-a,245-1-3-b
2015091506023,"Heinlein, Sylvia",NaN,2015,"barnebøker, lettlest",nno,2015,NaN,NaN,NaN
2015091506024,"Markhus, Vegard",Epleslang,2015,"barnebøker, barnelitteratur, lettlest, norsk",nno,2015,NaN,NaN,NaN
2018020248200,"Ganeri, Anita",Jorden på 2 minutter,cop. 2015,NaN,nob,2015,NaN,NaN,NaN
2018020548069,"Hofstad, Gunnleik",Venner for alltid :,2015,NaN,nob,2015,"en bok for tenåringer om relasjoner, kjærlighe...",NaN,NaN
2018050448035,"Gaarder, Jostein",Sofies verden,2015,NaN,nob,2015,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2018040948111,NaN,NaN,2015,"internett, personrett",nob,2015,NaN,NaN,NaN
2018050748058,NaN,NaN,2015,NaN,mul,2015,NaN,NaN,NaN
2018061807581,NaN,NaN,cop. 2015,NaN,nob,2015,NaN,NaN,NaN
2018062607029,NaN,NaN,cop. 2015,NaN,nob,2015,NaN,NaN,NaN


In [54]:
writer = pd.ExcelWriter('boklister_oppdatert_tema.xlsx', engine='xlsxwriter')
for i in ordgrupper:
    meta_bulitt.loc[[str(x) for x in corpus_text.loc[check_theme('seksualitet').index].dropna(axis=1, how='all').columns]].to_excel(writer, sheet_name=i)
writer.save()

### Lager en liste med konkordanser

Listen er basert på ordene i den tematiske oversikten. De bør gås gjennom

In [75]:
konks = lambda w, urner = barnebok_urns:[mm.konk(word = w, urns = urner, before=12, after=12)]

In [76]:
tema_ord = [w for t in temaer for w in temaer[t]]

In [77]:
konkordanser = {w:konks(w) for w in tema_ord}

In [79]:
link = lambda urn: "https://urn.nb.no/URN:NBN:no-nb_digibok_" + str(urn)

In [84]:
konks = []
for w in tema_ord:
    konks += [(link(x[0]), x[1], x[2], x[3]) for x in konkordanser[w][0]]

In [86]:
konk_df = nb.frame(konks)
konk_df.columns = ['url', 'venstre', 'tema', 'høyre']
konk_df

,url,venstre,tema,høyre
0,https://urn.nb.no/URN:NBN:no-nb_digibok_201712...,"frøken Strobe « Hikk , » sa Gregor . « Jeg er",blind,! »
1,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,Det er her ordtaket « kjærlighet gjør,blind,» kommer fra .
2,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,"hadde gått en time , og Siv hadde fanget fire ...",blind,"rotte og noe som så ut som en mark med tenner ,"
3,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,"Det var da hun skjønte det , Knuterotta var",blind,.
4,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,en eller annen grunn hadde hun ikke regnet med...,blind,knuterotte .
...,...,...,...,...
4654,https://urn.nb.no/URN:NBN:no-nb_digibok_201906...,"Kåfjord . Hun snakket sjøsamisk kebabnorsk , s...",samisk,", kvensk og norsk . - Due kiper betyr at du er"
4655,https://urn.nb.no/URN:NBN:no-nb_digibok_201906...,Plutselig kom CSV-erne stormende mot senteret ...,samisk,"ånd , og gruppen består av suksessrike politis..."
4656,https://urn.nb.no/URN:NBN:no-nb_digibok_201906...,", og gruppen består av suksessrike politisk ko...",samisk,"språk , kultur , kunst , næring og retten til ..."
4657,https://urn.nb.no/URN:NBN:no-nb_digibok_201805...,"lapon m [ lapa ] , iaponne / [ lapon ] adj",samisk,"Lapon m [ lapa ] , Laponne / [ lapon ] s"


In [89]:
konk_df[konk_df.tema == 'blind']

,url,venstre,tema,høyre
0,https://urn.nb.no/URN:NBN:no-nb_digibok_201712...,"frøken Strobe « Hikk , » sa Gregor . « Jeg er",blind,! »
1,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,Det er her ordtaket « kjærlighet gjør,blind,» kommer fra .
2,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,"hadde gått en time , og Siv hadde fanget fire ...",blind,"rotte og noe som så ut som en mark med tenner ,"
3,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,"Det var da hun skjønte det , Knuterotta var",blind,.
4,https://urn.nb.no/URN:NBN:no-nb_digibok_201802...,en eller annen grunn hadde hun ikke regnet med...,blind,knuterotte .
...,...,...,...,...
247,https://urn.nb.no/URN:NBN:no-nb_digibok_201806...,Det var en gang en,blind,"nordmann , en døv danske og en rullestol-svens..."
248,https://urn.nb.no/URN:NBN:no-nb_digibok_201807...,B : Hunden til en,blind,person C : Hunden til en trikkefører
249,https://urn.nb.no/URN:NBN:no-nb_digibok_201807...,7. B - Hunden til en,blind,9. B-Hest
250,https://urn.nb.no/URN:NBN:no-nb_digibok_201811...,som den jeg var ment til å være . Når jeg er,blind,", ser du klart . Når andre er døve , hører du"


In [90]:
konkwriter = pd.ExcelWriter('konkordanser.xlsx', engine='xlsxwriter')
for w in tema_ord:
    konk_df[konk_df.tema == w].to_excel(konkwriter, sheet_name = w)
konkwriter.save()

## Lag katalog over ordlistene

In [91]:
babi = nb.frame(corpus_text.fillna(0).sum(axis=1))

In [92]:
babi

,0
.,1376003.0
/,27236.0
",",845801.0
',25191.0
i,326948.0
...,...
Es-ra,1.0
Cmi,1.0
BteMMiw,1.0
AaJ,1.0


In [93]:
nb.frame(babi).to_csv('babi_ordliste.csv', header=None)

In [94]:
oversikt ={key: nb.frame_sort(
    nb.frame(
        {w:int(babi.loc[w].sum()) for w in temaer[key] if w in babi.index },
        'frekv'
    )
) for key in temaer.keys()}

In [103]:
oversikt_trunkert ={key: nb.frame_sort(
    nb.frame(
        {w:int(babi.loc[[ws for ws in babi.index if ws.startswith(w)]].sum()) for w in temaer[key] },
        'frekv'
    )
) for key in temaer.keys()}

# Oversikt over frekvens av ord

Her vil vi også kunne lage dokumentfrekvens ganske greit. "Om litt" :)

In [95]:
oversikt['diff']

,frekv
tynn,966
tykk,545
blind,321
døv,87
svaksynt,6
tunghørt,6
albino,3


In [96]:
oversikt['seksualitet']

,frekv
bi,76
homofil,67
transe,56
homse,15
lesbisk,8
bifil,4
lesbe,3
trans,2


In [97]:
oversikt['nasjonalitet']

,frekv
russisk,108
arabisk,78
polsk,46
afrikansk,34
amerikaner,27
russer,10
islandsk,9
hindu,5
asiat,5
pakistansk,2


In [98]:
oversikt['religion']

,frekv
islam,65
kristendom,21
jødedom,11
hinduisme,4
koptisk,4
taoisme,3
katolisisme,2


In [99]:
oversikt['geografi']

,frekv
Europa,504
Afrika,364
Amerika,200
Australia,192
Asia,131
Antarktis,101
Arktis,35
Norden,22
Ural,3


In [100]:
oversikt['nasjonalitet']

,frekv
russisk,108
arabisk,78
polsk,46
afrikansk,34
amerikaner,27
russer,10
islandsk,9
hindu,5
asiat,5
pakistansk,2


In [101]:
oversikt['språk']

,frekv
engelsk,586
fransk,320
tysk,190
svensk,130
russisk,108
italiensk,106
spansk,103
kinesisk,90
arabisk,78
japansk,55


In [102]:
oversikt['kulturmarkør']

,frekv
amerikaner,27
nordisk,14
samisk,14
indianer,12
araber,7
europeer,4
afrikaner,3


In [104]:

writer = pd.ExcelWriter('oversikt_temaer.xlsx', engine='xlsxwriter')

for i in oversikt:
    oversikt[i].to_excel(writer, sheet_name = i)

writer.save()

In [105]:

writer = pd.ExcelWriter('oversikt_temaer_trunkert.xlsx', engine='xlsxwriter')

for i in oversikt_trunkert:
    oversikt_trunkert[i].to_excel(writer, sheet_name = i)

writer.save()

In [106]:
mm.konk(word= 'urfolk' , urns = barnebok_urns, before = 10, after = 10)

[]

In [110]:
meta_bulitt.loc[[str(x) for x in corpus_text.loc['transe'].dropna().index]]

,100-1- -a,245-1-0-a,260- - -c,653- - -a,lang,year,245-1-0-b,245-1-3-a,245-1-3-b
2018060807125,"Shannon, Samantha",Mimeordenen,cop. 2015,"fantasy, romaner, ungdomsbøker",nob,2015,NaN,NaN,NaN
2018060807141,"Loughead, Deb",Trommer og trøbbel,2015,NaN,nob,2015,NaN,NaN,NaN
2018061107516,"Riordan, Rick",Hades' hus,cop. 2015,NaN,nob,2015,NaN,NaN,NaN
2018061807565,"McClintock, Norah.",Hvem er den skyldige?,2015,NaN,nob,2015,NaN,NaN,NaN
2018061907620,"Riordan, Rick",Athenes merke,2015,"barnebøker, fantastisk, fantasy, gresk mytolog...",nob,2015,NaN,NaN,NaN
2018062607117,"Stephens, John",Ildkrøniken,2015,NaN,nob,2015,NaN,NaN,NaN
2018062807021,"Smith, Dan",Big game :,2015,ungdomsbøker,nob,2015,jakten er i gang,NaN,NaN
2018062807068,"Walliams, David",Demontannlegen,2015,"barnebøker, barnelitteratur, humor, romaner, t...",nob,2015,NaN,NaN,NaN
2018070507023,"Sævareid, Heidi",Slipp hold,2015,"kjærlighet, kontroll, makt, skjønnlitteratur, ...",nob,2015,NaN,NaN,NaN
2018070507056,"Collins, Suzanne",Underland-krøniken,2015,NaN,nob,2015,NaN,NaN,NaN


In [112]:
excelwriter = pd.ExcelWriter('litteratur_gruppert_tema.xlsx', engine='xlsxwriter')
for t in ordgrupper:
    for w in temaer[t]:
        try:
            meta_bulitt.loc[[str(x) for x in corpus_text.loc[w].dropna().index]].to_excel(excelwriter, sheet_name=w)
        except:
            print(w)
excelwriter.save()

hørselshemmet
klumpfot
grønlender
grønlandsk
grønlandsk
castillano
tamil
muhammedanisme
urfolk
